In [1]:
import sys
import os, re, json, random, itertools, time
from os.path import join
import pandas as pd
tools_path = join(os.getcwd(), 'customer_behaviour/tools')
sys.path.insert(1, tools_path)
import policy_evaluation as pe
from result import Result
from tools import save_plt_as_eps, save_plt_as_png
import gym
import custom_gym
import chainer
import chainerrl
import seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

dir_path = 'ozzy_results/dummy/2020-05-08_14-50-32'

model_path = None # join(os.getcwd(), dir_path, '12288000_checkpoint', 'model.npz')

sample_length = 5000
normalize = True
n_demos_per_expert = 1
n_last_days = 7
max_n_purchases_per_n_last_days = 2
show_info = True
show_plots = False
save_plots = True
cluster_comparison = False

args_path = join(dir_path, 'args.txt')
args = json.loads(open(args_path, 'r').read())

info = pe.get_info(args)

# Get path of model 
model_dir_path = next((d for d in [x[0] for x in os.walk(dir_path)] if d.endswith('finish')), None)

os.makedirs(join(dir_path, 'figs'), exist_ok=True)

ending_eps = '_normalize.eps' if normalize else '.eps'
ending_png = '_normalize.png' if normalize else '.png'

In [2]:
from chainerrl.misc.batch_states import batch_states
def sample_probs_and_actions_from_policy(env, model, obs_normalizer, initial_state=None):
    xp = np
    phi = lambda x: x

    probs = []
    actions = []

    obs = env.reset().astype('float32')  # Initial state

    if initial_state is not None:
        env.state = initial_state
        obs = np.array(initial_state).astype('float32')

    done = False
    while not done:
        b_state = batch_states([obs], xp, phi)
        
        if obs_normalizer:
            b_state = obs_normalizer(b_state, update=False)

        with chainer.using_config('train', False), chainer.no_backprop_mode():
            action_distrib, _ = model(b_state)
            action = chainer.cuda.to_cpu(action_distrib.sample().array)[0]

        probs.append(action_distrib.all_prob.data[0][-1])
        actions.append(action)
        new_obs, _, done, _ = env.step(action)
        obs = new_obs.astype('float32')

    return probs, actions

In [4]:

def get_probs_for_buys(args, model_dir_path, expert=1, init='ones', dummi=True):
    env, model, obs_normalizer = pe.get_env_and_model(args, model_dir_path, sample_length, model_path=model_path)
    n_input_neurons = model.pi.model.in_size
    dummy = [0]*args['n_experts']
    if dummi:
        dummy[expert] = 1
    expert_trajectories = env.generate_expert_trajectories(out_dir=None, n_demos_per_expert=1, n_expert_time_steps=sample_length)
    expert_states = expert_trajectories['states']
    expert_actions = expert_trajectories['actions']
    e_actions = expert_actions[expert]
    e_states = expert_states[expert]
    if init == 'expert':
        index = np.random.randint(len(e_states))
        initial_state = e_states[index].tolist()
    elif init == 'ones':
        initial_state = dummy + [1]*(n_input_neurons-args['n_experts'])
    elif init == 'zeros':
        initial_state = dummy + [0]*(n_input_neurons-args['n_experts'])
    elif init == 'rand':
        from customer_behaviour.tools.dgm import DGM
        dgm = DGM()
        #dgm.spawn_new_customer(np.random.randint(11,100))
        dgm.spawn_new_customer(12)
        sample = dgm.sample((n_input_neurons-args['n_experts']))
        sample = np.sum(sample, axis=0)
        initial_state = np.concatenate((dummy,sample))
    probs, actions = sample_probs_and_actions_from_policy(env, model, obs_normalizer, initial_state=initial_state)
    return probs, actions, e_actions, expert_actions

probs, actions, e_actions, all_expert_actions = get_probs_for_buys(args, model_dir_path, expert=1, init='expert', dummi=False)
#plt.plot(probs)
#x1,x2,y1,y2 = plt.axis()
#plt.axis((x1,x2,0,1))
#plt.show()

"""Verkar indikera att agenten inte använder all information den får tillgång till. [1]*100 ger mer extremt resultat --> än [0]*100 eftersom den inte 'ser' hela sekvensen. """


"Verkar indikera att agenten inte använder all information den får tillgång till. [1]*100 ger mer extremt resultat --> än [0]*100 eftersom den inte 'ser' hela sekvensen. "

In [6]:
def get_purchase_ratio(sequence):
    if len(sequence) == 0:
        return np.count_nonzero(sequence)/1
    else: 
        seq=sequence[int(0.3*len(sequence)):]
        return np.count_nonzero(seq)/len(seq)

purchase_ratios = [get_purchase_ratio(actions[:i]) for i in range(len(actions))]
e_purchase_ratios = [get_purchase_ratio(e_actions[:i]) for i in range(len(e_actions))]
print(purchase_ratios[-1])
print(e_purchase_ratios[-1])

#temp = []
#for e in range(len(all_expert_actions)):
#    temp.append([get_purchase_ratio(all_expert_actions[e][:i]) for i in range(len(all_expert_actions[e]))])

#temp = np.array(temp)
#mean_purchase_ratio = np.mean(temp, axis=0)    

#cut_off = 300
#y = purchase_ratios[cut_off:]
#e_y = e_purchase_ratios[cut_off:]
#mean = mean_purchase_ratio[cut_off:]
#fig = plt.figure()
#plt.plot(range(cut_off, len(y) + cut_off), y, label='Agent')
#plt.plot(range(cut_off, len(e_y) + cut_off), e_y, label='Expert 2')
#plt.plot(range(cut_off, len(mean) + cut_off), mean, label='Average expert')
#plt.legend()
#plt.grid()
#plt.ylabel('Purchase ratio')
#plt.xlabel('Days')
#plt.title('GAIL')

#plt.show()

0.13857142857142857
0.1362857142857143


In [ ]:
zeros, dummy=true
rand, dummy=False
expertstate, dummy=false


In [ ]:
from customer_behaviour.tools.tools import save_plt_as_eps
save_plt_as_eps(fig, os.getcwd() + '/zeros_dummy.eps')

In [ ]:
def main():
    n_runs = 100
    for _ in range(n_runs):
        probs, actions, e_actions, all_expert_actions = get_probs_for_buys(args, model_dir_path, expert=2, init='zeros', dummi=True)
        purchase_ratios = [get_purchase_ratio(actions[:i]) for i in range(len(actions))]




